In [1]:
from itertools import combinations, product, chain
import numpy as np

In [2]:
def parse_input(filename: str) -> np.ndarray:
    row_lst = []
    with open(filename) as f:
        for row in f:
            row = row.rstrip()
            row_lst.append(list(map(int, row.split(","))))
    return np.stack(row_lst)

In [3]:
def num_covered_sides(input_arr: np.ndarray) -> int:
    counter = 0
    for a, b in combinations(input_arr, 2):
        if (a - b).astype(bool).sum() <= 1 and abs((a - b).sum()) == 1:
            counter += 1

    return counter


def surface_area(input_arr: np.ndarray, check_closed_in: bool = False) -> int:
    num_of_sides = 6 * len(input_arr)
    num_of_sides -= 2 * num_covered_sides(input_arr)

    if check_closed_in:
        min_x, min_y, min_z = input_arr.min(axis=0)
        max_x, max_y, max_z = input_arr.max(axis=0)

        cube_set = set(map(tuple, input_arr))
        closed_in_cubes = 0
        for x, y, z in product(
            range(min_x, max_x + 1),
            range(min_y, max_y + 1),
            range(min_y, max_z + 1),
        ):
            if (x, y, z) in cube_set:
                continue
            neighbor_set = {
                ((x - 1), y, z),
                ((x + 1), y, z),
                (x, (y - 1), z),
                (x, (y + 1), z),
                (x, y, (z - 1)),
                (x, y, (z + 1)),
            }
            if neighbor_set.issubset(cube_set):
                closed_in_cubes += 1
        num_of_sides -= 6 * closed_in_cubes

    return num_of_sides

In [4]:
surface_area(np.array([[1, 1, 1], [2, 1, 1]]))

10

In [5]:
input_arr = parse_input("test-input.txt")
surface_area(input_arr)

64

In [6]:
input_arr = parse_input("input.txt")
surface_area(input_arr)

3522

In [7]:
from tqdm import tqdm


def graph_based_pocket_detection(input_arr: np.ndarray) -> int:
    import networkx as nx

    graph = nx.Graph()
    min_x, min_y, min_z = input_arr.min(axis=0)
    max_x, max_y, max_z = input_arr.max(axis=0)
    cube_set = set(map(tuple, input_arr))

    for x, y, z in product(
        range(min_x, max_x + 1),
        range(min_y, max_y + 1),
        range(min_z, max_z + 1),
    ):
        graph.add_node((x, y, z), droplet=(x, y, z) in cube_set)

    full_cubes = set(graph.nodes)

    for a in graph.nodes:
        a_arr = np.array(a)
        for diff in [
            (1, 0, 0), (-1, 0, 0), (0, 1, 0), (0, -1, 0), (0, 0, 1), (0, 0, -1),
        ]:
            diff = np.array(diff)
            b = tuple(a_arr + diff)
            if b in full_cubes:
                graph.add_edge(a, b, weight=1 if a in cube_set or b in cube_set else 0)

    print(graph)

    outer_set = set(
        chain.from_iterable(
            [
                (x, y, min_z),
                (x, y, max_z),
                (x, min_y, z),
                (x, max_y, z),
                (min_x, y, z),
                (max_x, y, z),
            ]
            for x, y, z in graph.nodes
        )
    ).difference(cube_set)

    non_real_cubes = set(graph.nodes).difference(cube_set)
    num_of_trapped = 0
    for cube in tqdm(non_real_cubes):
        try:
            dist, path = nx.multi_source_dijkstra(
                graph,
                sources=outer_set,
                target=cube,
                cutoff=0,
            )
        except nx.NetworkXNoPath:
            for diff in [
                (1, 0, 0), (-1, 0, 0), (0, 1, 0), (0, -1, 0), (0, 0, 1), (0, 0, -1),
            ]:
                b = tuple(np.array(cube) + np.array(diff))
                if b in cube_set:
                    num_of_trapped += 1
    return num_of_trapped

In [8]:
input_arr = parse_input("test-input.txt")
trapped_faces = graph_based_pocket_detection(input_arr)
area = surface_area(input_arr)
print(area - trapped_faces)

Graph with 54 nodes and 117 edges


100%|██████████| 41/41 [00:00<00:00, 49316.45it/s]

58


In [9]:
input_arr = parse_input("input.txt")
trapped_faces = graph_based_pocket_detection(input_arr)
area = surface_area(input_arr)
print(area - trapped_faces)

Graph with 7600 nodes and 21640 edges


100%|██████████| 5557/5557 [01:23<00:00, 66.64it/s] 


2074
